# 1.Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm
import matplotlib
import os
import json
import requests

# 2. Investigating Ratings

In [ ]:
r_cols = ["user_id", "item_id", "rating", "timestamp"]
ratings_df = pd.read_csv("data/ml-1m/ratings.dat", engine="python", sep="::", names=r_cols)
ratings_df.head()

In [ ]:
# Basic statistics
num_users = ratings_df["user_id"].nunique()
num_items = ratings_df["item_id"].nunique()
print(f"Số lượng users: {num_users}")
print(f"Số lượng items: {num_items}")

interaction_count = len(ratings_df)
print(f"Số lượng tương tác user-item: {interaction_count}")

In [ ]:
# # avg rating per user
# plt.figure(figsize=(7,5))
# ratings_per_user = ratings_df.groupby('user_id').size()
# ratings_per_user_filtered = ratings_per_user[ratings_per_user <= 500]
# sns.histplot(ratings_per_user_filtered, bins=25, kde=False)
# plt.title("Phân phối số lượng rating mỗi user (0 - 500)")
# plt.xlabel("Số rating mỗi user")
# plt.ylabel("Số lượng user")
# plt.xlim(0, 500)
# plt.show()

In [ ]:
# # avg rating per item
# plt.figure(figsize=(7,5))
# ratings_per_item = ratings_df.groupby('item_id').size()
# ratings_per_item_filtered = ratings_per_item[ratings_per_item <= 500]
# sns.histplot(ratings_per_item_filtered, bins=25, kde=False)
# plt.title("Phân phối số lượng rating mỗi item (0 - 500)")
# plt.xlabel("Số rating mỗi item")
# plt.ylabel("Số lượng item")
# plt.xlim(0, 500)
# plt.tight_layout()
# plt.show()

In [ ]:
# Tính rating density (utility matrix density)
density = interaction_count / (num_users * num_items)
print(f"Rating density (utility matrix density): {density:.6f}")

# plt.figure(figsize=(6,4))
# sns.countplot(x='rating', data=ratings_df)
# plt.title("Phân phối giá trị rating")
# plt.xlabel("Rating")
# plt.ylabel("Số lượng rating")
# plt.show()

In [ ]:
avg_rating_per_item = ratings_df.groupby('item_id')['rating'].mean()
avg_rating_per_user = ratings_df.groupby('user_id')['rating'].mean()

print(f"Trung bình rating mỗi item: \n{avg_rating_per_item.head()}")
print(f"Trung bình rating mỗi user: \n{avg_rating_per_user.head()}")

In [ ]:
# # Optional: Vẽ biểu đồ trung bình rating mỗi user và mỗi item
# plt.figure(figsize=(14,5))

# plt.subplot(1,2,1)
# sns.histplot(avg_rating_per_user, bins=50, kde=True)
# plt.title("Phân phối trung bình rating mỗi user")
# plt.xlabel("Trung bình rating")
# plt.ylabel("Số lượng user")

# plt.subplot(1,2,2)
# sns.histplot(avg_rating_per_item, bins=50, kde=True)
# plt.title("Phân phối trung bình rating mỗi item")
# plt.xlabel("Trung bình rating")
# plt.ylabel("Số lượng item")

# plt.tight_layout()
# plt.show()

In [ ]:
ratings_df.to_csv("ratings.csv", index=False)

# 3. Investigating Users

In [ ]:
u_cols = ["user_id", "gender", "age", "occupation", "zip_code"]
user_df = pd.read_csv("data/ml-1m/users.dat", engine="python", sep="::", names=u_cols)

user_df.head()

In [ ]:
user_df.to_csv("users.csv", index=False)

# 4. Investigating Movies

In [ ]:
m_cols = ["item_id", "title", "genre"]
movies_df = pd.read_csv(
    "data/ml-1m/movies.dat",
    sep="::",
    engine="python",
    names=m_cols,
    encoding="latin-1"         
)

movies_df.head()

In [ ]:
# Function to compute avg rating per movie 
def compute_avg_ratings(df):
    agg = (
        df.groupby("item_id")["rating"]
        .agg(avg_rating="mean", num_ratings="count")
        .reset_index()
    )
    return agg

avg_df = compute_avg_ratings(ratings_df)

In [ ]:
movie_titles = movies_df["title"].tolist()
movie_titles[0:5]

In [ ]:
# import os
# import json
# import requests
# from tqdm import tqdm
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import re

# # ── Config ─────────────────────────────────────────────────────────────────────
# TMDB_API_KEY = "4d8b37c0931a16ee9390065936018eac"
# SEARCH_URL   = "https://api.themoviedb.org/3/search/movie"
# DETAILS_URL  = "https://api.themoviedb.org/3/movie/{movie_id}"

# OUTPUT_FILE = "descriptions.jsonl"
# LOG_FILE    = "error_log.txt"

# # ── Helpers ────────────────────────────────────────────────────────────────────
# YEAR_RE = re.compile(r"^(.*)\s+\((\d{4})\)\s*$")
# def split_title_and_year(full_title):
#     m = YEAR_RE.match(full_title)
#     if m:
#         return m.group(1), int(m.group(2))
#     return full_title, None

# def fetch_description_for_title(full_title):
#     name, year = split_title_and_year(full_title)
#     try:
#         params = {
#             "api_key":      TMDB_API_KEY,
#             "query":        name,
#             "include_adult": False
#         }
#         if year:
#             params["primary_release_year"] = year

#         r = requests.get(SEARCH_URL, params=params, timeout=5)
#         r.raise_for_status()
#         results = r.json().get("results", [])
#         if not results:
#             raise ValueError("no-results")

#         movie_id = results[0]["id"]
#         r2 = requests.get(
#             DETAILS_URL.format(movie_id=movie_id),
#             params={"api_key": TMDB_API_KEY, "language": "en-US"},
#             timeout=5
#         )
#         r2.raise_for_status()

#         overview = r2.json().get("overview", "")
#         return (full_title, overview, None)
#     except Exception as e:
#         return (full_title, "", str(e))

# results = []
# with open(OUTPUT_FILE, "w", encoding="utf-8") as fout, \
#      open(LOG_FILE,    "w", encoding="utf-8") as flog, \
#      ThreadPoolExecutor(max_workers=5) as pool:

#     futures = { pool.submit(fetch_description_for_title, t): t for t in movie_titles }

#     for fut in tqdm(as_completed(futures), total=len(futures), desc="Fetching 10"):
#         title = futures[fut]
#         t, overview, err = fut.result()

#         # 1) Log error if any
#         if err:
#             flog.write(f"{t} - {err}\n")

#         # 2) Write out to JSONL
#         record = {"title": t, "description": overview}
#         fout.write(json.dumps(record, ensure_ascii=False) + "\n")

#         # 3) Collect for printing
#         results.append(record)

# print("\nFetched Descriptions (first 100 chars):")
# for rec in results:
#     print(f"- {rec['title']}: {rec['description'][:100]!r}")

In [ ]:
desc_df = pd.read_json("descriptions.jsonl", lines=True)[["title", "description"]]

In [ ]:
desc_df.head()

In [ ]:
movies_stats = movies_df.merge(
    avg_df,
    on="item_id",
    how="left"
) 

movies_full = movies_stats.merge(
    desc_df,
    on="title",
    how="left"
)

movies_full["avg_rating"]  = movies_full["avg_rating"].fillna(0)
movies_full["num_ratings"] = movies_full["num_ratings"].fillna(0).astype(int)
movies_full["description"] = movies_full["description"].fillna("")

movies_full = movies_full[
    ["item_id", "title", "genre", "avg_rating", "num_ratings", "description"]
]

movies_full.to_csv("movies.csv", index=False, encoding="utf-8")
print("Wrote", len(movies_full), "rows to movies.csv")